<h1> Wildebeest Detection using U-Net from VHR satellite images</h1>
Code Author: Zijing Wu 

***The code is developed for research project purposes.***

In [ ]:
#If you are using Google Colaboratory to run this code, please upload the whole folder to your Google Drive, and run this cell install the requirements.

#connect to the google drive if you use Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')

#install the libraries
!pip install rasterio
!pip install geopandas

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 308 kB/s 
     |████████████████████████████████| 6.3 MB 27.9 MB/s 


#Preparation

##  Load libraries

In [ ]:
import os
from osgeo import gdal

import rasterio
from rasterio import windows
from rasterio.windows import Window

from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2
import matplotlib.pyplot as plt

!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 245 µs (started: 2022-07-21 12:18:06 +00:00)


## Define functions

In [ ]:
def generate_mask(raster_path, shape_path, output_path, file_name):
    
    """Function that generates a binary mask from a vector file (shp or geojson)
    
    raster_path = path to the .tif;

    shape_path = path to the shapefile or GeoJson.

    output_path = Path to save the binary mask.

    file_name = Name of the file.
    
    """
    
    #load raster
    
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    
    #load o shapefile ou GeoJson
    train_df = gpd.read_file(shape_path)
    
        
    #Function that generates the mask
    def poly_from_utm(polygon, transform):
        poly_pts = []

        poly = cascaded_union(polygon)
        for i in np.array(poly.exterior.coords):

            poly_pts.append(~transform * tuple(i))

        new_poly = Polygon(poly_pts)
        return new_poly
    
    
    poly_shp = []
    im_size = (src.meta['height'], src.meta['width'])
    for num, row in train_df.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], src.meta['transform'])
            poly_shp.append(poly)
        else:
            for p in row['geometry']:
                poly = poly_from_utm(p, src.meta['transform'])
                poly_shp.append(poly)

    mask = rasterize(shapes=poly_shp,
                     out_shape=im_size)
    
    #Salve
    mask = mask.astype("uint8")
    
    bin_mask_meta = src.meta.copy()
    bin_mask_meta.update({'count': 1})
    os.chdir(output_path)
    with rasterio.open(file_name, 'w', **bin_mask_meta) as dst:
        dst.write(mask, 1)

time: 32.5 ms (started: 2022-07-21 12:19:16 +00:00)


In [ ]:
def generate_empty_mask(raster_path, output_path, file_name):
    
    """Function that generates an empty binary mask (all 0) from a vector file (shp or geojson). Used for patches where there is no target object.
    
    raster_path = path to the .tif;

    shape_path = path to the shapefile or GeoJson.

    output_path = Path to save the binary mask.

    file_name = Name of the file.
    
    """
    
    #load raster
    
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    im_size = (src.meta['height'], src.meta['width'])
    
    mask = np.zeros(im_size)
    
    #Save
    mask = mask.astype("uint8")
    
    bin_mask_meta = src.meta.copy()
    bin_mask_meta.update({'count': 1})
    os.chdir(output_path)
    with rasterio.open(file_name, 'w', **bin_mask_meta) as dst:
        dst.write(mask, 1)

time: 11.5 ms (started: 2022-07-21 12:20:15 +00:00)


## Define the data path

In [ ]:
#Change the path to your directory

#For Google Colaboratory users, update the directory:
Data_folder = "/content/drive/MyDrive/Colab/Wildebeest-UNet/SampleData/1_Data_preparation/"
#For Jupyter Notebook users, update the directory:
#Data_folder = "Wildebeest-UNet/SampleData/1_Data_preparation"

IMAGE_PATH = os.path.join(Data_folder, "image")
ROI_PATH = os.path.join(Data_folder, "roi")
MASK_PATH = os.path.join(Data_folder, "mask")

time: 3.24 ms (started: 2022-07-21 12:40:48 +00:00)


# Processing

## Create the mask images from the source image and annotation AOIs

In [ ]:
for f in sorted(os.listdir(IMAGE_PATH)):
    #print(f)
    fdir = os.path.join(IMAGE_PATH, f)
    image_name, ext = os.path.splitext(f)
    if ext.lower() == ".tif":
        ID = image_name
        print(ID)

        match_status = 0
        for f in sorted(os.listdir(ROI_PATH)):
            roi_dir = os.path.join(ROI_PATH, f)
            mask_name, ext = os.path.splitext(f)
            if ext.lower() == ".shp":
                roi_ID = mask_name
                print(roi_ID)
                

                if roi_ID == ID:
                    generate_mask(fdir, roi_dir, MASK_PATH, ID+'.tif')
                    print("Generated FN mask image " + ID)
                    match_status = 1
                    
        if match_status == 0:
            
            generate_empty_mask(fdir, MASK_PATH, ID+'.tif')
            print("Generated FP mask image " + ID)               

TR_10
TR29
Generated FP mask image TR_10
TR_29
TR29
Generated FP mask image TR_29
time: 729 ms (started: 2022-07-21 12:40:54 +00:00)


# References


***References:***

https://lpsmlgeo.github.io/2019-09-22-binary_mask/